In [43]:
from math import cos, sin, pi

In [44]:
class Car:
    def __init__(self):
        """
        Инициализация автомобиля.
        Устанавливаются начальные значения для скорости, угла и координат.
        Также создается пустой список для команд управления.
        """
        self.speed = 0
        self.angle = 0
        self.x = 0
        self.y = 0
        self.commands = []
    
    def move(self, time):
        """
        Перемещение автомобиля.
        Скорость и угол используются для вычисления смещения автомобиля по осям x и y
        """
        self.x += self.speed * time * cos(self.angle)
        self.y += self.speed * time * sin(self.angle)

    def set_speed(self, speed):
        """
        Обновление скорости автомобиля.
        """
        self.speed = speed
        
    def set_angle(self, angle):
        """
        Обновление угла движения автомобиля.
        """
        self.angle = angle * pi / 180  # Преобразование угла в радианы для вычислений
    
    def add_command(self, time, speed=None, angle=None):
        """
        Добавление новой команды в список команд. 
        Команды могут включать время выполнения, скорость и угол движения.
        После добавления новой команды список команд сортируется по времени.
        """        
        if self.commands and time < self.commands[-1]["time"]:
            raise ValueError("Новая команда не можт быть добавлена, так как она находится после последнего действия в последовательности")

        self.commands.append({"time": time, "speed": speed, "angle": angle})
        self.commands.sort(key=lambda x: x['time'])  # Сортировка для надежности, но теперь она не требуется

    def validate_commands_sequence(self):
        """
        Валидируем корректность последовательности комманд по времени.
        """
        for i in range(len(self.commands) - 1):
            if self.commands[i]['time'] > self.commands[i+1]['time']:
                return False
        return True

    def get_coordinates_at_time(self, target_time):
        """
        Вычисляем параметры машины в произвольный момент времени.
        """
        current_time, last_speed, last_angle = 0, self.speed, self.angle

        for command in self.commands:
            if target_time < command['time']:
                break

            time_delta = command['time'] - current_time
            self.move(time_delta)

            if command['speed'] is not None:
                self.set_speed(command['speed'])
            if command['angle'] is not None:
                self.set_angle(command['angle'])
            
            current_time = command['time']
            last_speed, last_angle = self.speed, self.angle

        # Завершить оставшееся движение до target_time
        time_delta = target_time - current_time
        if time_delta > 0:
            self.x += last_speed * time_delta * cos(last_angle)
            self.y += last_speed * time_delta * sin(last_angle)
        
        return self.x, self.y

    def execute_commands(self):
        """
        Запускает выполнение команд.
        Команды выполняются в порядке в котором они находятся в списке (после сортировки по времени).
        После выполнения всех команд функция возвращает текущие координаты автомобиля.
        """
        return self.get_coordinates_at_time(self.commands[-1]['time']) if self.commands else (self.x, self.y)

In [45]:
# Пример 1: Создаем автомобиль и задаем ему последовательность движений
my_car = Car()
my_car.add_command(0, speed=10)  # начинаем движение с скоростью 10
my_car.add_command(5, angle=pi/2)  # на 5-й секунде поворачиваем на 90 градусов
my_car.add_command(7, speed=0)  # на 7-й секунде останавливаемся
print(my_car.execute_commands())  # выводим координаты, где остановился автомобиль

(69.99248433718964, 0.5482426718408858)


In [46]:
# Пример 2: Создаем автомобиль и задаем ему последовательность движений
car = Car()
car.add_command(0, speed=10)      
car.add_command(5, speed=20)      
car.add_command(10, angle=90)    

assert car.validate_commands_sequence() == True

In [47]:
try:
    car.add_command(2, angle=180)  # Добавляем команду с временем раньше последней
except ValueError as errr:
    print(errr)

Новая команда не можт быть добавлена, так как она находится после последнего действия в последовательности


In [48]:
assert car.validate_commands_sequence() == True

In [49]:
# Пример 3: Управляем несколькими автомобилями
car1 = Car()
car1.add_command(0, speed=5)
car1.add_command(3, angle=pi)
car1.add_command(6, speed=0)

car2 = Car()
car2.add_command(0, speed=7)
car2.add_command(2, angle=pi/2)
car2.add_command(4, speed=10)
car2.add_command(6, angle=pi)
car2.add_command(7, speed=0)

print(car1.execute_commands())  
print(car2.execute_commands()) 

(29.977457247957958, 0.8220549772318428)
(57.972194871861014, 1.4800491936174012)


In [50]:
#Пример 4: Достаем параметры в произвольное время
car = Car()
car.add_command(0, speed=10)  
car.add_command(5, speed=0)  

# Произвольное время до остановки
x_before_stop, y_before_stop = car.get_coordinates_at_time(3)
print(f"Координаты (x, y) на 3-й секунде: ({x_before_stop}, {y_before_stop})")

# Произвольное время после остановки на 6 секунде
x_after1_stop, y_after1_stop = car.get_coordinates_at_time(6)
print(f"Координаты (x, y) на 6-й секунде: ({x_after1_stop}, {y_after1_stop})")

# Произвольное время после остановки на 10 секунде (не длжно измениться)
x_after2_stop, y_after2_stop = car.get_coordinates_at_time(10)
print(f"Координаты (x, y) на 10-й секунде: ({x_after_stop}, {y_after_stop})")

Координаты (x, y) на 3-й секунде: (30.0, 0.0)
Координаты (x, y) на 6-й секунде: (80.0, 0.0)
Координаты (x, y) на 10-й секунде: (80.0, 0.0)


In [51]:
car = Car()
car.add_command(0, speed=15)  
car.add_command(5, angle=90)
car.add_command(10, speed=0) 

final_x, final_y = car.execute_commands()
print(f"Конечные координаты (x, y): ({final_x}, {final_y})")
# Первые 5 секунд на север: x = 0 (cos(0) * 0), y = 15 * 5 (sin(0) * 15 * 5)
# Следующие 5 секунд на восток: x = 0 + 15 * 5 (cos(pi/2) * 15 * 5)
# Конечные координаты x = 75, y = 75

Конечные координаты (x, y): (75.0, 75.0)
